In [1]:
from pathlib import Path
import json
import socket

import pandas as pd
import geopandas as gpd

from pyagnps import soil_data_market as sdm

from sqlalchemy import URL, create_engine, text as sql_text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

In [2]:
# DATABASE SETUP
credentials = Path("../../inputs/db_credentials.json")
with open(credentials, "r") as f:
    credentials = json.load(f)

user = credentials["user"]
password = credentials["password"]
host = credentials["host"]
port = credentials["port"]
database = credentials["database"]

url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

# create a SQLAlchemy engine object
engine = create_engine(url_object)

nodename = socket.gethostname()

In [3]:
raster_CDL_path = Path('D:/AIMS/Datasets/Management/CDL_Annual/CDL_2022.tif')
path_to_management_class_names = Path('../../inputs/management/CDL_All_Codes.csv')

In [4]:
df_cdl = pd.read_csv(path_to_management_class_names)

In [5]:
df_cdl.head()

,Value,Class_Name,Count2021,Src_Year,Value1,RGB_R,RGB_G,RGB_B,Comments
0,0,Background,0,2021.0,0,0.0,0.0,0.0,"Added 0 so the background presents as null, no..."
1,1,Corn,427057895,2021.0,1,255.0,211.0,0.0,NaN
2,2,Cotton,57641094,2021.0,2,255.0,37.0,37.0,NaN
3,3,Rice,12260890,2021.0,3,0.0,168.0,226.0,NaN
4,4,Sorghum,40371933,2021.0,4,255.0,158.0,9.0,NaN


In [6]:
dico = df_cdl[['Value','Class_Name']].set_index('Value').to_dict(orient='dict')['Class_Name']
# dico = df_cdl.set_index('CDL_Value').loc[:,['Mgmt_Schd_ID']].to_dict()
dico = {key: '' if isinstance(value,float) else value for key, value in dico.items()}

In [8]:
classes = set(dico.values())

# Remove classes that need 
already_valid_classes = classes - set(["", "Clouds_No_Data", "Developed"])

reversed_already_valid_dico = {dico[val]: val for val in dico if dico[val] in already_valid_classes}

In [64]:
thuc_id = '2092'

- Loop through T-HUCs
  - Create `CDL_Value_2022` column
  - Populate with matching CDL Value for known categories i.e. anything different than "NaN", "Developed", "Clouds_No_Data" and give -1 to the three mentioned

In [65]:
invalid_field_ids_to_fix = ("'NaN'", "'Clouds_No_Data'", "'Developed'")

with engine.connect() as connection:
    try:
        # Make sure that mgmt_field_id column is indeed text 
        # create a column for cdl value 2022
        # if the mgmt field values are NULL, NaN, Clouds_No_Data, or Developed the cdl_value is set to -1
        query = f"""
        ALTER TABLE thuc_{thuc_id}_annagnps_cell_data_section
        ALTER COLUMN mgmt_field_id SET DATA TYPE text USING CAST(mgmt_field_id AS text);

        ALTER TABLE thuc_{thuc_id}_annagnps_cell_data_section
        ADD COLUMN IF NOT EXISTS cdl_value_2022 INT;

        UPDATE thuc_{thuc_id}_annagnps_cell_data_section 
        SET cdl_value_2022 = CASE 
            WHEN mgmt_field_id IS NULL OR mgmt_field_id IN ({', '.join(invalid_field_ids_to_fix)}) THEN -1 
            ELSE NULL 
        END
        """

        connection.execute(sql_text(query))
        connection.commit()

    except Exception as e:
        print(f"Error for THUC {thuc_id}")
        print(e)
        connection.rollback()

Find mgmt_field_id that have a cdl_value_2022 different than -1 we apply the mapping

In [98]:
# query = 'SELECT * FROM usa_valid_soil_layers_data WHERE "Soil_ID" = 568326'
query = f"SELECT mgmt_field_id FROM thuc_{thuc_id}_annagnps_cell_data_section WHERE cdl_value_2022 <> -1 OR cdl_value_2022 IS NULL"

df = pd.read_sql_query(sql=sql_text(query), con=engine.connect())
df = df.drop_duplicates().reset_index(drop=True) # Find the unique classes present in that THUC

In [99]:
with engine.connect() as connection:
    try:

        query = f"""
        UPDATE thuc_{thuc_id}_annagnps_cell_data_section
        SET cdl_value_2022 = CASE WHEN mgmt_field_id = :Mgmt_Field_ID THEN :CDL_Value_2022 ELSE -1 END
        WHERE mgmt_field_id = :Mgmt_Field_ID
        """
       
        for _, class_name in df.iterrows():
            name = class_name['mgmt_field_id']
            connection.execute(sql_text(query), {"CDL_Value_2022": reversed_already_valid_dico[name], "Mgmt_Field_ID": name})

        connection.commit()

    except Exception as e:
        print(f"Error for THUC {thuc_id}")
        print(e)
        connection.rollback()

Query remaining cells and apply zonal statistics to only those

In [101]:
# Get cell_ids
query = f"SELECT cell_id FROM thuc_{thuc_id}_annagnps_cell_data_section WHERE cdl_value_2022 = -1 OR cdl_value_2022 IS NULL"

df = pd.read_sql_query(sql=sql_text(query), con=engine.connect())

In [106]:
cells_ids_to_reprocess = tuple(df['cell_id'].to_list())

In [108]:
# Collect thuc cells geometry from database

query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_ids WHERE dn in {cells_ids_to_reprocess}"

with engine.connect() as conn:
    cells = gpd.read_postgis(sql=sql_text(query), con=conn, geom_col="geom")

utm = cells.estimate_utm_crs()
cells = cells.to_crs(utm)


Doing the plurality analysis using the raster

In [111]:
cells = sdm.assign_attr_zonal_stats_raster_layer(cells, raster_CDL_path, agg_method='majority', attr='CDL_Value')
cells['CDL_Value'] = cells['CDL_Value'].astype('Int32')
# cells.loc[cells['CDL_Value']==0, 'CDL_Value'] = 81 # Set 0 value to 81 = Cloud_No_Data
cells['Mgmt_Field_ID'] = cells['CDL_Value'].map(dico)
cells = cells.rename(columns={"dn": "cell_id"})

c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\rasterstats\io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


In [112]:
cells

,fid,cell_id,geom,CDL_Value,Mgmt_Field_ID
0,8137,32863,"POLYGON ((597200.579 3601315.075, 597200.579 3...",228,Dbl_Crop_Triticale_Corn
1,8860,22733,"POLYGON ((592400.579 3600625.075, 592400.579 3...",61,Fallow_Idle_Cropland
2,8874,32882,"POLYGON ((597410.579 3601075.075, 597410.579 3...",228,Dbl_Crop_Triticale_Corn
3,9151,22733,"POLYGON ((591830.579 3599665.075, 591830.579 3...",226,Dbl_Crop_Oats_Corn
4,9177,22733,"POLYGON ((591800.579 3599635.075, 591800.579 3...",61,Fallow_Idle_Cropland
...,...,...,...,...,...
392,25765,2482,"POLYGON ((598430.579 3581305.075, 598430.579 3...",228,Dbl_Crop_Triticale_Corn
393,25789,2482,"POLYGON ((598400.579 3581275.075, 598400.579 3...",228,Dbl_Crop_Triticale_Corn
394,25859,2482,"POLYGON ((598340.579 3581245.075, 598340.579 3...",228,Dbl_Crop_Triticale_Corn
395,25887,2513,"POLYGON ((598370.579 3581515.075, 598370.579 3...",228,Dbl_Crop_Triticale_Corn


In [113]:
cells.explore(column='Mgmt_Field_ID', categorical=True)

In [15]:
data_to_update = cells[["cell_id", "Mgmt_Field_ID"]].to_dict(orient="records")

Set mgmt_field_id colum as text

In [16]:
with engine.connect() as connection:
    try:
        query = f"ALTER TABLE thuc_{thuc_id}_annagnps_cell_data_section ALTER COLUMN mgmt_field_id TYPE TEXT"
        connection.execute(sql_text(query))
        # Commit the transaction explicitly
        connection.commit()

    except Exception as e:
        print(e)
        # Rollback the transaction in case of an error
        connection.rollback()


Populate Data

In [17]:
try:
    # create a session factory
    Session = sessionmaker(bind=engine)
    # create a new session
    session = Session()
    # create a transaction
    transaction = session.begin()

    # execute your update query here
    query = f"UPDATE thuc_{thuc_id}_annagnps_cell_data_section SET Mgmt_Field_ID = :Mgmt_Field_ID WHERE cell_id = :cell_id"
    session.execute(sql_text(query), data_to_update)
    # commit the transaction
    transaction.commit()

except Exception as e:
    # rollback the transaction on error
    transaction.rollback()

finally:
    # close the session
    session.close()

Check that it worked

In [33]:
# query = 'SELECT * FROM usa_valid_soil_layers_data WHERE "Soil_ID" = 568326'
query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_data_section ORDER BY cell_id"

df = pd.read_sql_query(sql=sql_text(query), con=engine.connect())
df.head()

,cell_id,soil_id,mgmt_field_id,reach_id,reach_location_code,cell_area,time_of_conc,avg_elevation,rcn_calib_id,avg_land_slope,...,shallow_conc_flow_length,delivery_ratio,constant_usle_c_fctr,constant_usle_p_fctr,all_oc_calib_fctr,all_n_calib_fctr,all_p_calib_fctr,sheet_and_rill_erosion_calib_fctr,gullies_erosion_calib_fctr,input_units_code
0,22,295611,None,2,1,54.18,None,203.59,None,0.27312,...,50.0,None,None,None,None,None,None,None,None,1
1,23,295681,None,2,1,30.42,None,199.45,None,0.41944,...,50.0,None,None,None,None,None,None,None,None,1
2,32,295678,None,3,1,5.40,None,256.17,None,0.05635,...,50.0,None,None,None,None,None,None,None,None,1
3,33,295575,None,3,1,4.14,None,255.01,None,0.05885,...,50.0,None,None,None,None,None,None,None,None,1
4,41,295632,None,4,0,11.07,None,275.41,None,0.03250,...,50.0,None,None,None,None,None,None,None,None,1


In [24]:
df[df['mgmt_field_id'].isnull()].empty

True